In [ ]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from __M1model__ import glob_all, define_generator, define_discriminator
from __M1model__ import define_composite_model_A2B, define_composite_model_B2A, train

from __M1_TargetEmbedding__ import Encoder, Decoder, M1_1_AEModel
from tensorflow.keras import backend as K
from __init__ import glob_all

import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().disabled = True

import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.config.optimizer.set_jit(True)

In [ ]:
channel_number = 6; resnet_n = 9 ; epochs = 100; batch = 1

dataA_norm = np.load('new_dataA_norm.npy')
dataB_norm = np.load('new_dataB_norm.npy')
print('Loaded', dataA_norm.shape, dataB_norm.shape)

EPA73_testing_PM25 = np.load('EPA73_Testing_348X204_PM25.npy', mmap_mode='r')

station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
extract5_list = np.load('RandomTesting/extract5_list.npy')
extract10_list = np.load('RandomTesting/extract10_list.npy')
extract15_list = np.load('RandomTesting/extract15_list.npy')
extract20_list = np.load('RandomTesting/extract20_list.npy')

ex20_KNNhalf = np.load('ex20_KNN_half.npy', mmap_mode='r')
ex20_KNN_norm = np.load('ex20_KNN_norm.npy', mmap_mode='r')

model_path = 'M1_1_Graph_RMSprop_Res'+str(resnet_n)+'_CycleGan/'
if not os.path.exists(model_path+'result/'):os.makedirs(model_path+'result/')

In [ ]:
K.clear_session()
image_shape = dataB_norm[0].shape

TE_Encoder_model = Encoder(channel_number)
TE_Decoder_model = Decoder(channel_number)
x = Input(shape=(348, 204, 1))
TE_Autoencoder = Model(x,TE_Decoder_model(TE_Encoder_model(x)))
TE_weight = glob_all('Unpair_Target_Embedding_AIRSAT_channel'+str(channel_number)+'_K3')[0]

TE_Autoencoder.load_weights(TE_weight)
for layer in TE_Autoencoder.layers:layer.trainable = False
    
M1_1_Inmodel = M1_1_AEModel(channel_number)
M1_1_Outmodel = M1_1_AEModel(channel_number)
g_model_AtoB = Model(x,M1_1_Outmodel(TE_Autoencoder(M1_1_Inmodel(x))))
print('generator: A -> B')

g_model_BtoA = define_generator(image_shape,resnet_n)
print('generator: B -> A')
d_model_A = define_discriminator(image_shape, d_lr = 5e-5)
print('discriminator: A -> [real/fake]')
d_model_B = define_discriminator(image_shape, d_lr = 5e-5)
print('discriminator: B -> [real/fake]')
c_model_AtoB = define_composite_model_A2B(g_model_AtoB, d_model_B, g_model_BtoA, image_shape, g_lr = 1e-3)
print('composite: A -> B -> [real/fake, A]')
c_model_BtoA = define_composite_model_B2A(g_model_BtoA, d_model_A, g_model_AtoB, image_shape, g_lr = 1e-3)
print('composite: B -> A -> [real/fake, B]')

train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA,
      dataA_norm, dataB_norm, model_path, EPA73_testing_PM25,
      station_coordinate, extract5_list, extract10_list, extract15_list, extract20_list, 
      ex20_KNN_norm, ex20_KNNhalf, n_epochs=epochs, n_batch=batch, save_epochs = 1)

print('train models done')